# Carto test. 
(https://gist.github.com/jorgesancha/2a8027e5a89a2ea1693d63a45afdd8b6)

Goals:
+ Read a large file.
+ Count number of lines.
+ Calculate averge value for 'tip_amount' field.

All code was run on a slow 2011 Mac book air with 4gb memory and 1.7ghz i5 processor. 

First a baseline for comparison.

In [1]:
%time !wc -l yellow_tripdata_2016-01.csv

 10906859 yellow_tripdata_2016-01.csv
CPU times: user 235 ms, sys: 64.7 ms, total: 299 ms
Wall time: 8.38 s


So we have a base line of around 8 secs for iterating over the file for a line count. Let's take a quick look at the columns we're dealing with and find where tip_amount is.

In [2]:
header = !head -1 yellow_tripdata_2016-01.csv 
header

['VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount']

In [3]:
header[0].split(',').index('tip_amount') #column for tip_amount, 0 indexed

15

Now we can do a quick check to compute the mean of the tip_amount field. Above we saw it was the 15th field, but that was zero indexed. We'll use awk to compute a quick value for the mean (we're calculating the sum of tip_amount and the number of lines. From that we return the mean).

In [4]:
%time !awk -F ',' '{sum+=$16; lines++} END { print lines, sum/lines}' yellow_tripdata_2016-01.csv

10906859 1.75066
CPU times: user 3.76 s, sys: 1.13 s, total: 4.89 s
Wall time: 2min 51s


So this little awk one liner takes about 3 minutes to compute the number of lines and the mean for tip_amount.

Can we do better with Python?

Let's start with a simple function to return the line count.

In [5]:
def line_count(in_file):
    line_count = 0
    with open(in_file) as file_data:
        for row in file_data:
            line_count += 1
        return line_count

In [6]:
%time line_count(in_file='yellow_tripdata_2016-01.csv')

CPU times: user 10.1 s, sys: 1.3 s, total: 11.4 s
Wall time: 11.8 s


10906859

So a little slower than wc, but let's add in a few more lines to handle calculating the mean and see how it handles.

In [7]:
def line_count_mean(in_file, column_index):
    """Reads a file and computes line count and mean of a column.
    args:
        file to read <string>, column index to compute average of <int>.
    return:
        <tuple>: line count and average of passed in column."""
    line_count = 0
    column_sum = 0
    with open(in_file, 'r') as file_data:
        next(file_data) # skip header
        for row in file_data:
            line_count += 1
            column_sum += float(row.split(',')[column_index])
        
        column_average = column_sum / line_count
        return (line_count, round(column_average,2))
            

In [8]:
# First let's run it once and make sure it works:
print(line_count_mean(in_file='yellow_tripdata_2016-01.csv', column_index=15))

(10906858, 1.75)


Looks good. Note, the line count is off by once since we skip the header.

In [9]:
%timeit line_count_mean(in_file='yellow_tripdata_2016-01.csv', column_index=15)

37.1 s ± 1.72 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Huge improvement over awk. Using the with/open syntax (context manager) takes care of the file management part. Also, since Python 3 the for loop there now uses a generator behind the scenes, so each line is lazily evaluated and then garbage collected (huge savings on memory).

Now let's take a quick look at Pandas and see how it compares.

In [10]:
# Is pandas any faster than plain python3?
import pandas as pd
# importing the file into memory will one potential bottleneck.
%time df = pd.read_csv('yellow_tripdata_2016-01.csv')

CPU times: user 50.2 s, sys: 15.2 s, total: 1min 5s
Wall time: 1min 11s


In [11]:
df.shape #runs instantly

(10906858, 19)

In [12]:
%timeit df['tip_amount'].mean()

97 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Pandas is considerably faster, but that's beause it loads the data into memory first. That actually takes a far amount of time and is slower than our solution above. Although once in memory we can run many tests a lot faster.

One more interesting thing to try would be to see if we can speed up pd.read_csv(). 

In [13]:
%time df_c = pd.read_csv('yellow_tripdata_2016-01.csv', engine='c')

CPU times: user 51 s, sys: 16.7 s, total: 1min 7s
Wall time: 1min 25s


In [14]:
%timeit df_c['tip_amount'].mean()

104 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%time df_mm = pd.read_csv('yellow_tripdata_2016-01.csv', memory_map=True)

CPU times: user 50.5 s, sys: 18.3 s, total: 1min 8s
Wall time: 1min 34s


In [16]:
%timeit df_mm['tip_amount'].mean()

90.7 ms ± 8.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Slightly faster on computation, but still slow on I/O. 

# Conclusion:

Streaming a file line by line and computing statistics as you go is the most efficient way to deal with large files. Using Pandas allows for more analysis to be done, but comes at the cost of loading the file into memory.

A couple of further ideas:
+ rewriting the looping and summing code in C. 
+ Consider storing the file data in a database, HDF,  or any non flat format that may speed up I/O.

In [17]:
def line_count_mean(in_file, column_index):
    """Reads a file and computes line count and mean of a column.
    args:
        file to read <string>, column index to compute average of <int>.
    return:
        <tuple>: line count and average of passed in column."""
    line_count = 0
    column_sum = 0
    with open(in_file, 'r') as file_data:
        next(file_data) # skip header
        for row in file_data:
            line_count += 1
            column_sum += float(row.split(',')[column_index])
        
        column_average = column_sum / line_count
        return (line_count, round(column_average,2))